In [1]:
import pandas as pd
import numpy as np

# Demographic Variables and Sample Weights (DEMO_H)
RIAGENDR - Gender
RIDAGEYR - Age in years at screening
female = 2

# Medical Conditions (MCQ_H)
SEQN - Respondent sequence number
- MCQ230A - What kind of cancer (Code 14 is Breast, many codes many different cancers)
- MCQ240E - Age when breast cancer first diagnosed
- MCQ220 - Ever told you had cancer or malignancy
no = 2


# Complete Blood Count with 5-part Differential - Whole Blood (CBC_H)
SEQN - Respondent sequence number
LBXWBCSI - White blood cell count (1000 cells/uL)
LBXLYPCT - Lymphocyte percent (%)
LBXMOPCT - Monocyte percent (%)
LBXNEPCT - Segmented neutrophils percent (%)
LBXEOPCT - Eosinophils percent (%)
LBXBAPCT - Basophils percent (%)
LBDLYMNO - Lymphocyte number (1000 cells/uL)
LBDMONO - Monocyte number (1000 cells/uL)
LBDNENO - Segmented neutrophils num (1000 cell/uL)
LBDEONO - Eosinophils number (1000 cells/uL)
LBDBANO - Basophils number (1000 cells/uL)
LBXRBCSI - Red blood cell count (million cells/uL)
LBXHGB - Hemoglobin (g/dL)
LBXHCT - Hematocrit (%)
LBXMCVSI - Mean cell volume (fL)
LBXMCHSI - Mean cell hemoglobin (pg)
LBXMC - MCHC (g/dL)
LBXRDW - Red cell distribution width (%)
LBXPLTSI - Platelet count (1000 cells/uL)
LBXMPSI - Mean platelet volume (fL)
 

# Standard Biochemistry Profile (BIOPRO_H)
SEQN - Respondent sequence number
LBDSALSI - Albumin (g/L)
LBXSAPSI - Alkaline phosphatase (IU/L)
LBXSASSI - Aspartate aminotransferase AST (IU/L)
LBXSATSI - Alanine aminotransferase ALT (IU/L)
LBXSBU - Blood urea nitrogen (mg/dL)
LBXSC3SI - Bicarbonate (mmol/L)
LBDSCASI - Total calcium (mmol/L)
LBDSCHSI - Cholesterol (mmol/L)
LBXSCK - Creatine Phosphokinase(CPK) (IU/L)
LBXSCLSI - Chloride (mmol/L)
LBDSCRSI - Creatinine (umol/L)
LBDSGBSI - Globulin (g/L)
LBDSGLSI - Glucose, refrigerated serum (mmol/L)
LBXSGTSI - Gamma glutamyl transferase (U/L)
LBDSIRSI - Iron, refrigerated serum (umol/L)
LBXSKSI - Potassium (mmol/L)
LBXSLDSI - Lactate dehydrogenase (U/L)
LBXSNASI - Sodium (mmol/L)
LBXSOSSI - Osmolality (mmol/Kg)
LBDSPHSI - Phosphorus (mmol/L)
LBDSTBSI - Total bilirubin (umol/L)
LBDSTPSI - Total protein (g/L)
LBDSTRSI - Triglycerides, refrigerated (mmol/L)
LBDSUASI - Uric acid (umol/L)

# GET MORE DATA

In [2]:
for i in ['A']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[(df_med['Breast Cancer'] == 1)]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')

    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final = df.copy()

In [3]:
for i in ['B','C','D','E','F','G','H','I']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[(df_med['Breast Cancer'] == 1)]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')


    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
   # 'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
   # 'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    #'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final = df_final.append(df)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [4]:
df_final['Breast Cancer'].value_counts()

1    656
Name: Breast Cancer, dtype: int64

# do the same for the other women

In [5]:
for i in ['A']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')
    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[df_med['Breast Cancer'] == 0]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')


    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    #'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final_h = df.copy()

In [6]:
for i in ['B','C','D','E','F','G','H','I']:    
    df_demo= pd.read_sas('./NHANES 2013-2014/other NHANES/DEMO_'+i+'.XPT')

    # selecting specific columns
    # selecting gender: women SEQN and Age
    df_demo = df_demo[(df_demo['RIAGENDR'] == 2 ) & (df_demo['RIDAGEYR'] > 10) ][['SEQN', 'RIDAGEYR']]
    df_demo = df_demo.rename(columns={'RIDAGEYR': 'Age'})

    df_med = pd.read_sas('./NHANES 2013-2014/other NHANES/MCQ_'+i+'.XPT')
    # match only dose SEQN matching the one inf df_demo (seqn variable)
    df_med = df_med.loc[df_med['SEQN'].isin(df_demo['SEQN'])]
    df_med = df_med[['SEQN', 'MCQ230A', 'MCQ240E', 'MCQ220']]
    # add column breast cancer 'Diagnosis' 1 no 0
    df_med['Breast Cancer'] = np.where(df_med['MCQ230A'] == 14,1,0)

    # subset to have women that were diagnosed with breast cancer 
    df_med = df_med[df_med['Breast Cancer'] == 0]
    # changing the column name to diagnosis age and saving the dataset
    df_med = df_med.rename(columns={'MCQ240E' : 'Diagnosis Age'})[['SEQN', 'Diagnosis Age', 'Breast Cancer']]
    # merge df_med with df_demo
    df = pd.merge(df_med, df_demo, on='SEQN', how='inner')
    
    
    df_blood = pd.read_sas('./NHANES 2013-2014/other NHANES/CBC_'+i+'.XPT')
    # subset to SEQN from df
    df_blood = df_blood.loc[df_blood['SEQN'].isin(df['SEQN'])]
    col_dict = {
    'LBXWBCSI' : 'BL_White blood cell count (1000 cells/uL)',
    'LBXLYPCT' : 'BL_Lymphocyte percent (%)',
    'LBXMOPCT' : 'BL_Monocyte percent (%)',
    'LBXNEPCT' : 'BL_Segmented neutrophils percent (%)',
    'LBXEOPCT' : 'BL_Eosinophils percent (%)',
    'LBXBAPCT' : 'BL_Basophils percent (%)',
    'LBDLYMNO' : 'BL_Lymphocyte number (1000 cells/uL)',
    'LBDMONO' : 'BL_Monocyte number (1000 cells/uL)',
    'LBDNENO' : 'BL_Segmented neutrophils num (1000 cell/uL)',
    'LBDEONO' : 'BL_Eosinophils number (1000 cells/uL)',
    'LBDBANO' : 'BL_Basophils number (1000 cells/uL)',
    'LBXRBCSI' : 'BL_Red blood cell count (million cells/uL)',
    'LBXHGB' : 'BL_Hemoglobin (g/dL)',
    'LBXHCT' : 'BL_Hematocrit (%)',
    'LBXMCVSI' : 'BL_Mean cell volume (fL)',
 #   'LBXMCHSI' : 'BL_Mean cell hemoglobin (pg)',
 #   'LBXMC' : 'BL_MCHC (g/dL)',
    'LBXRDW' : 'BL_Red cell distribution width (%)',
    'LBXPLTSI' : 'BL_Platelet count (1000 cells/uL)',
    'LBXMPSI' : 'BL_Mean platelet volume (fL)'
    }
    df_blood = df_blood.rename(columns = col_dict)
    # merge df_blood to df
    df = pd.merge(df, df_blood, how='inner', on=['SEQN'])

    df_bio = pd.read_sas('./NHANES 2013-2014/other NHANES/BIOPRO_'+i+'.XPT')
    col_dict = {
    'SEQN' : 'SEQN',
    'LBDSALSI' : 'BIO_Albumin (g/L)',
    #'LBXSAPSI' : 'BIO_Alkaline phosphatase (IU/L)',
    'LBXSASSI' : 'BIO_Aspartate aminotransferase AST (IU/L)',
    'LBXSATSI' : 'BIO_Alanine aminotransferase ALT (IU/L)',
    'LBXSBU' : 'BIO_Blood urea nitrogen (mg/dL)',
    #'LBDSBUSI' : 'BIO_Blood urea nitrogen (mmol/L)',
    'LBXSC3SI' : 'BIO_Bicarbonate (mmol/L)',
    'LBDSCASI' : 'BIO_Total calcium (mmol/L)',
    'LBDSCHSI' : 'BIO_Cholesterol (mmol/L)',
    #'LBXSCK' : 'BIO_Creatine Phosphokinase(CPK) (IU/L)',
    'LBXSCLSI' : 'BIO_Chloride (mmol/L)',
    'LBDSCRSI' : 'BIO_Creatinine (umol/L)',
    'LBDSGBSI' : 'BIO_Globulin (g/L)',
    'LBDSGLSI' : 'BIO_Glucose, refrigerated serum (mmol/L)',
    'LBXSGTSI' : 'BIO_Gamma glutamyl transferase (U/L)',
    'LBDSIRSI' : 'BIO_Iron, refrigerated serum (umol/L)',
    'LBXSKSI' : 'BIO_Potassium (mmol/L)',
   # 'LBXSLDSI' : 'BIO_Lactate dehydrogenase (U/L)',
    'LBXSNASI' : 'BIO_Sodium (mmol/L)',
    'LBXSOSSI' : 'BIO_Osmolality (mmol/Kg)',
    'LBDSPHSI' : 'BIO_Phosphorus (mmol/L)',
    'LBDSTBSI' : 'BIO_Total bilirubin (umol/L)',
    'LBDSTPSI' : 'BIO_Total protein (g/L)',
    'LBDSTRSI' : 'BIO_Triglycerides, refrigerated (mmol/L)',
    'LBDSUASI' : 'BIO_Uric acid (umol/L)'}
    # select the columns i want to keep
    df_bio = df_bio[list(col_dict.keys())].rename(columns=col_dict)
    # merge df_bio with df
    df = pd.merge(df, df_bio, how='inner', on=['SEQN'])
    
    df_final_h = df_final_h.append(df)

In [7]:
df_final_h['Breast Cancer'].value_counts()

0    31503
Name: Breast Cancer, dtype: int64

In [8]:
df_final_final = df_final.append(df_final_h)

In [9]:
df_final_final['Breast Cancer'].value_counts()

0    31503
1      656
Name: Breast Cancer, dtype: int64

# Clean up

# create 2 subsets from women that were diagnosed. one for biochem one for blood 

In [10]:
df = df_final_final.copy()
# clean column names
df.columns = df.columns.str.lower().str.capitalize()
#rename the Seqn column
df = df.rename(columns={'Seqn':'Id'})

In [11]:
df.head()

,Age,Bio_alanine aminotransferase alt (iu/l),Bio_albumin (g/l),Bio_aspartate aminotransferase ast (iu/l),Bio_bicarbonate (mmol/l),Bio_blood urea nitrogen (mg/dl),Bio_chloride (mmol/l),Bio_cholesterol (mmol/l),Bio_creatinine (umol/l),Bio_gamma glutamyl transferase (u/l),...,Bl_red cell distribution width (%),Bl_segmented neutrophils num (1000 cell/ul),Bl_segmented neutrophils percent (%),Bl_white blood cell count (1000 cells/ul),Breast cancer,Diagnosis age,Lbxmc,Lbxmch,Lbxmchsi,Id
0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,73.0,NaN,NaN,NaN,80.0
1,61.0,21.0,40.0,21.0,25.0,15.0,101.7,6.08,53.0,22.0,...,12.9,2.8,45.3,6.1,1,56.0,32.8,NaN,30.5,94.0
2,50.0,23.0,45.0,25.0,27.0,16.0,101.8,6.28,61.9,28.0,...,13.4,3.3,58.3,5.6,1,44.0,32.8,NaN,28.2,206.0
3,81.0,13.0,45.0,19.0,23.0,17.0,104.0,6.78,44.2,20.0,...,12.3,3.4,53.3,6.4,1,75.0,34.2,NaN,31.5,237.0
4,81.0,27.0,42.0,32.0,30.0,18.0,104.0,4.60,61.9,30.0,...,13.3,3.1,62.7,4.9,1,76.0,34.3,NaN,31.9,389.0


# BIOCHEM

In [12]:
# column names for subset BIO
df_bio_col = ['Id', 'Age']
for i in list(df.columns):
    if i.startswith('Bio'):
        df_bio_col.append(i)
df_bio_col.append('Breast cancer')

df_bio = df[df_bio_col]

In [13]:
df_bio

,Id,Age,Bio_alanine aminotransferase alt (iu/l),Bio_albumin (g/l),Bio_aspartate aminotransferase ast (iu/l),Bio_bicarbonate (mmol/l),Bio_blood urea nitrogen (mg/dl),Bio_chloride (mmol/l),Bio_cholesterol (mmol/l),Bio_creatinine (umol/l),...,Bio_osmolality (mmol/kg),Bio_phosphorus (mmol/l),Bio_potassium (mmol/l),Bio_sodium (mmol/l),Bio_total bilirubin (umol/l),Bio_total calcium (mmol/l),Bio_total protein (g/l),"Bio_triglycerides, refrigerated (mmol/l)",Bio_uric acid (umol/l),Breast cancer
0,80.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,94.0,61.0,21.0,40.0,21.0,25.0,15.0,101.7,6.080,53.00,...,277.0,1.227,4.13,138.3,5.10,2.275,71.0,2.269,285.5,1
2,206.0,50.0,23.0,45.0,25.0,27.0,16.0,101.8,6.280,61.90,...,281.0,1.130,4.16,140.9,8.60,2.325,74.0,1.106,267.7,1
3,237.0,81.0,13.0,45.0,19.0,23.0,17.0,104.0,6.780,44.20,...,287.0,1.130,4.64,143.5,12.00,2.400,77.0,1.039,243.9,1
4,389.0,81.0,27.0,42.0,32.0,30.0,18.0,104.0,4.600,61.90,...,289.0,1.162,3.97,144.6,23.90,2.450,68.0,1.219,279.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,93686.0,13.0,14.0,43.0,21.0,23.0,8.0,105.0,4.707,50.39,...,273.0,1.485,4.06,138.0,6.84,2.400,70.0,1.558,267.7,0
3385,93687.0,16.0,17.0,48.0,22.0,23.0,7.0,109.0,4.396,82.21,...,279.0,1.356,4.08,141.0,8.55,2.325,74.0,0.700,237.9,0
3386,93689.0,69.0,52.0,39.0,44.0,24.0,20.0,107.0,4.189,51.27,...,283.0,1.292,3.98,141.0,6.84,2.300,66.0,0.813,285.5,0
3387,93697.0,80.0,16.0,44.0,25.0,22.0,16.0,99.0,6.801,81.33,...,269.0,1.292,3.93,134.0,5.13,2.350,66.0,1.095,273.6,0


In [14]:
df_bio_bcancer = df_bio[df_bio['Breast cancer'] == 1]
df_bio_nbcancer = df_bio[df_bio['Breast cancer'] == 0]

In [15]:
df_bio_bcancer.isna().sum()

Id                                            0
Age                                           0
Bio_alanine aminotransferase alt (iu/l)      90
Bio_albumin (g/l)                            90
Bio_aspartate aminotransferase ast (iu/l)    90
Bio_bicarbonate (mmol/l)                     90
Bio_blood urea nitrogen (mg/dl)              90
Bio_chloride (mmol/l)                        90
Bio_cholesterol (mmol/l)                     90
Bio_creatinine (umol/l)                      90
Bio_gamma glutamyl transferase (u/l)         90
Bio_globulin (g/l)                           90
Bio_glucose, refrigerated serum (mmol/l)     90
Bio_iron, refrigerated serum (umol/l)        90
Bio_osmolality (mmol/kg)                     90
Bio_phosphorus (mmol/l)                      90
Bio_potassium (mmol/l)                       90
Bio_sodium (mmol/l)                          90
Bio_total bilirubin (umol/l)                 90
Bio_total calcium (mmol/l)                   90
Bio_total protein (g/l)                 

In [16]:
# there are 90 values missing from many columns, lets just drop them
df_bio_bcancer = df_bio_bcancer.dropna(thresh=20)
df_bio_bcancer 

,Id,Age,Bio_alanine aminotransferase alt (iu/l),Bio_albumin (g/l),Bio_aspartate aminotransferase ast (iu/l),Bio_bicarbonate (mmol/l),Bio_blood urea nitrogen (mg/dl),Bio_chloride (mmol/l),Bio_cholesterol (mmol/l),Bio_creatinine (umol/l),...,Bio_osmolality (mmol/kg),Bio_phosphorus (mmol/l),Bio_potassium (mmol/l),Bio_sodium (mmol/l),Bio_total bilirubin (umol/l),Bio_total calcium (mmol/l),Bio_total protein (g/l),"Bio_triglycerides, refrigerated (mmol/l)",Bio_uric acid (umol/l),Breast cancer
1,94.0,61.0,21.0,40.0,21.0,25.0,15.0,101.7,6.080,53.00,...,277.0,1.227,4.13,138.3,5.10,2.275,71.0,2.269,285.5,1
2,206.0,50.0,23.0,45.0,25.0,27.0,16.0,101.8,6.280,61.90,...,281.0,1.130,4.16,140.9,8.60,2.325,74.0,1.106,267.7,1
3,237.0,81.0,13.0,45.0,19.0,23.0,17.0,104.0,6.780,44.20,...,287.0,1.130,4.64,143.5,12.00,2.400,77.0,1.039,243.9,1
4,389.0,81.0,27.0,42.0,32.0,30.0,18.0,104.0,4.600,61.90,...,289.0,1.162,3.97,144.6,23.90,2.450,68.0,1.219,279.6,1
5,481.0,63.0,43.0,41.0,11.0,24.0,10.0,101.5,3.490,35.40,...,280.0,1.195,3.60,140.1,6.80,2.325,71.0,1.084,208.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,93308.0,80.0,16.0,44.0,24.0,28.0,18.0,97.0,5.819,56.58,...,278.0,1.130,4.05,138.0,10.26,2.350,70.0,1.750,226.0,1
78,93374.0,50.0,12.0,40.0,19.0,27.0,14.0,104.0,6.284,60.11,...,279.0,1.518,3.58,140.0,10.26,2.275,70.0,0.745,267.7,1
79,93413.0,57.0,20.0,40.0,19.0,27.0,14.0,102.0,5.431,70.72,...,276.0,1.453,3.82,138.0,6.84,2.300,71.0,1.298,232.0,1
80,93568.0,46.0,22.0,44.0,17.0,20.0,16.0,107.0,6.077,59.23,...,285.0,1.033,3.92,143.0,1.71,2.400,76.0,3.681,279.6,1


In [17]:
df_bio_bcancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 1 to 81
Data columns (total 24 columns):
Id                                           566 non-null float64
Age                                          566 non-null float64
Bio_alanine aminotransferase alt (iu/l)      566 non-null float64
Bio_albumin (g/l)                            566 non-null float64
Bio_aspartate aminotransferase ast (iu/l)    566 non-null float64
Bio_bicarbonate (mmol/l)                     566 non-null float64
Bio_blood urea nitrogen (mg/dl)              566 non-null float64
Bio_chloride (mmol/l)                        566 non-null float64
Bio_cholesterol (mmol/l)                     566 non-null float64
Bio_creatinine (umol/l)                      566 non-null float64
Bio_gamma glutamyl transferase (u/l)         566 non-null float64
Bio_globulin (g/l)                           566 non-null float64
Bio_glucose, refrigerated serum (mmol/l)     566 non-null float64
Bio_iron, refrigerated serum (umol/l) 

In [18]:
# remove the NaNs from the set without breast cancer
df_bio_nbcancer = df_bio_nbcancer.dropna(thresh=20)
# there are 566 entries, subset random 566 from the women that were not diangosed
df_bio_nbcancer = df_bio_nbcancer.sample(n=566)

In [19]:
df_bio_nbcancer

,Id,Age,Bio_alanine aminotransferase alt (iu/l),Bio_albumin (g/l),Bio_aspartate aminotransferase ast (iu/l),Bio_bicarbonate (mmol/l),Bio_blood urea nitrogen (mg/dl),Bio_chloride (mmol/l),Bio_cholesterol (mmol/l),Bio_creatinine (umol/l),...,Bio_osmolality (mmol/kg),Bio_phosphorus (mmol/l),Bio_potassium (mmol/l),Bio_sodium (mmol/l),Bio_total bilirubin (umol/l),Bio_total calcium (mmol/l),Bio_total protein (g/l),"Bio_triglycerides, refrigerated (mmol/l)",Bio_uric acid (umol/l),Breast cancer
72,10175.0,21.0,16.0,41.0,21.0,19.0,11.0,107.0,2.900,53.04,...,275.0,1.160,3.50,138.0,10.26,2.225,73.0,0.294,285.5,0
3159,9117.0,56.0,17.0,40.0,26.0,23.0,18.0,107.5,4.910,53.00,...,282.0,0.969,4.09,140.6,5.10,2.650,73.0,1.445,350.9,0
2338,90582.0,50.0,22.0,49.0,24.0,27.0,14.0,104.0,6.568,42.43,...,282.0,1.356,3.91,142.0,15.39,2.350,73.0,2.574,368.8,0
2401,58473.0,57.0,18.0,44.0,21.0,28.0,16.0,104.0,5.793,85.75,...,280.0,1.292,4.00,140.0,15.39,2.400,78.0,0.926,410.4,0
519,53068.0,55.0,49.0,41.0,43.0,23.0,12.0,108.0,4.784,55.69,...,281.0,1.421,4.00,141.0,11.97,2.375,70.0,1.502,297.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3055,18883.0,13.0,29.0,44.0,26.0,22.0,12.0,101.0,5.840,70.72,...,276.0,1.740,4.40,139.0,10.26,2.525,81.0,1.400,315.2,0
1500,45857.0,80.0,13.0,39.0,23.0,26.0,9.0,103.0,5.767,65.42,...,285.0,0.969,3.30,143.0,13.68,2.475,74.0,1.558,398.5,0
2955,81991.0,12.0,18.0,43.0,20.0,27.0,9.0,104.0,4.758,45.08,...,275.0,1.647,4.40,139.0,8.55,2.475,71.0,2.506,291.5,0
2488,28215.0,60.0,13.0,44.0,18.0,28.0,13.0,102.0,6.336,88.40,...,279.0,1.227,3.60,140.0,13.68,2.375,69.0,1.976,458.0,0


In [20]:
df_bio_nbcancer.isna().sum()

Id                                           0
Age                                          0
Bio_alanine aminotransferase alt (iu/l)      0
Bio_albumin (g/l)                            0
Bio_aspartate aminotransferase ast (iu/l)    0
Bio_bicarbonate (mmol/l)                     0
Bio_blood urea nitrogen (mg/dl)              0
Bio_chloride (mmol/l)                        0
Bio_cholesterol (mmol/l)                     0
Bio_creatinine (umol/l)                      0
Bio_gamma glutamyl transferase (u/l)         0
Bio_globulin (g/l)                           0
Bio_glucose, refrigerated serum (mmol/l)     0
Bio_iron, refrigerated serum (umol/l)        0
Bio_osmolality (mmol/kg)                     0
Bio_phosphorus (mmol/l)                      0
Bio_potassium (mmol/l)                       0
Bio_sodium (mmol/l)                          0
Bio_total bilirubin (umol/l)                 0
Bio_total calcium (mmol/l)                   0
Bio_total protein (g/l)                      0
Bio_triglycer

# same for blood

In [21]:
# column names for subset BL ( blood )
df_bl_col = ['Id', 'Age']
for i in list(df.columns):
    if i.startswith('Bl'):
        df_bl_col.append(i)
df_bl_col.append('Breast cancer')

df_bl = df[df_bl_col]

In [22]:
df_bl

,Id,Age,Bl_basophils number (1000 cells/ul),Bl_basophils percent (%),Bl_eosinophils number (1000 cells/ul),Bl_eosinophils percent (%),Bl_hematocrit (%),Bl_hemoglobin (g/dl),Bl_lymphocyte number (1000 cells/ul),Bl_lymphocyte percent (%),...,Bl_mean platelet volume (fl),Bl_monocyte number (1000 cells/ul),Bl_monocyte percent (%),Bl_platelet count (1000 cells/ul),Bl_red blood cell count (million cells/ul),Bl_red cell distribution width (%),Bl_segmented neutrophils num (1000 cell/ul),Bl_segmented neutrophils percent (%),Bl_white blood cell count (1000 cells/ul),Breast cancer
0,80.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,94.0,61.0,5.397605e-79,0.6,1.000000e-01,0.9,35.6,11.6,2.7,43.9,...,8.6,0.6,9.3,322.0,3.84,12.9,2.8,45.3,6.1,1
2,206.0,50.0,5.397605e-79,0.7,1.000000e-01,1.6,36.7,12.0,1.9,33.8,...,7.3,0.3,5.7,301.0,4.27,13.4,3.3,58.3,5.6,1
3,237.0,81.0,5.397605e-79,0.6,1.000000e-01,1.9,41.8,14.3,2.2,34.3,...,8.2,0.6,9.8,287.0,4.53,12.3,3.4,53.3,6.4,1
4,389.0,81.0,1.000000e-01,1.1,2.000000e-01,3.7,38.7,13.3,1.1,21.7,...,8.7,0.5,10.9,191.0,4.16,13.3,3.1,62.7,4.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,93686.0,13.0,1.000000e-01,1.3,3.000000e-01,3.7,38.1,12.7,2.6,29.4,...,8.8,0.6,6.8,312.0,4.44,12.9,5.3,59.0,9.0,0
3385,93687.0,16.0,5.397605e-79,0.7,2.000000e-01,3.7,40.2,13.2,3.0,49.9,...,8.3,0.5,8.3,288.0,4.50,12.5,2.3,37.5,6.0,0
3386,93689.0,69.0,1.000000e-01,0.9,3.000000e-01,4.3,39.3,13.1,2.0,29.4,...,9.0,0.4,5.3,211.0,4.44,13.5,4.1,60.2,6.8,0
3387,93697.0,80.0,5.397605e-79,0.8,2.000000e-01,3.8,36.2,11.6,2.2,45.9,...,8.1,0.6,12.5,172.0,4.44,16.9,1.7,37.1,4.7,0


In [23]:
df_bl_bcancer = df_bl[df_bl['Breast cancer'] == 1]
df_bl_nbcancer = df_bl[df_bl['Breast cancer'] == 0]

In [24]:
df_bl_bcancer.isna().sum()

Id                                              0
Age                                             0
Bl_basophils number (1000 cells/ul)            73
Bl_basophils percent (%)                       73
Bl_eosinophils number (1000 cells/ul)          73
Bl_eosinophils percent (%)                     73
Bl_hematocrit (%)                              69
Bl_hemoglobin (g/dl)                           69
Bl_lymphocyte number (1000 cells/ul)           73
Bl_lymphocyte percent (%)                      73
Bl_mean cell volume (fl)                       69
Bl_mean platelet volume (fl)                   69
Bl_monocyte number (1000 cells/ul)             73
Bl_monocyte percent (%)                        73
Bl_platelet count (1000 cells/ul)              69
Bl_red blood cell count (million cells/ul)     69
Bl_red cell distribution width (%)             69
Bl_segmented neutrophils num (1000 cell/ul)    73
Bl_segmented neutrophils percent (%)           73
Bl_white blood cell count (1000 cells/ul)      69


In [25]:
df_bl_bcancer

,Id,Age,Bl_basophils number (1000 cells/ul),Bl_basophils percent (%),Bl_eosinophils number (1000 cells/ul),Bl_eosinophils percent (%),Bl_hematocrit (%),Bl_hemoglobin (g/dl),Bl_lymphocyte number (1000 cells/ul),Bl_lymphocyte percent (%),...,Bl_mean platelet volume (fl),Bl_monocyte number (1000 cells/ul),Bl_monocyte percent (%),Bl_platelet count (1000 cells/ul),Bl_red blood cell count (million cells/ul),Bl_red cell distribution width (%),Bl_segmented neutrophils num (1000 cell/ul),Bl_segmented neutrophils percent (%),Bl_white blood cell count (1000 cells/ul),Breast cancer
0,80.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,94.0,61.0,5.397605e-79,0.6,0.1,0.9,35.6,11.6,2.7,43.9,...,8.6,0.6,9.3,322.0,3.84,12.9,2.8,45.3,6.1,1
2,206.0,50.0,5.397605e-79,0.7,0.1,1.6,36.7,12.0,1.9,33.8,...,7.3,0.3,5.7,301.0,4.27,13.4,3.3,58.3,5.6,1
3,237.0,81.0,5.397605e-79,0.6,0.1,1.9,41.8,14.3,2.2,34.3,...,8.2,0.6,9.8,287.0,4.53,12.3,3.4,53.3,6.4,1
4,389.0,81.0,1.000000e-01,1.1,0.2,3.7,38.7,13.3,1.1,21.7,...,8.7,0.5,10.9,191.0,4.16,13.3,3.1,62.7,4.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,93308.0,80.0,1.000000e-01,0.9,0.5,4.8,41.4,13.8,2.6,23.3,...,10.1,0.7,6.4,215.0,4.62,15.1,7.1,64.7,11.0,1
78,93374.0,50.0,1.000000e-01,1.9,0.4,6.8,34.2,11.0,1.9,34.2,...,8.3,0.3,4.7,239.0,4.33,15.4,2.9,52.5,5.6,1
79,93413.0,57.0,1.000000e-01,1.0,0.1,0.9,42.5,13.8,1.8,23.8,...,8.0,0.6,8.0,238.0,4.39,12.8,5.0,66.4,7.5,1
80,93568.0,46.0,1.000000e-01,0.8,0.1,0.9,44.6,15.0,2.8,33.6,...,7.7,0.5,6.3,195.0,4.98,12.4,4.8,58.5,8.2,1


In [26]:
# many missing values
df_bl_bcancer = df_bl_bcancer.dropna(thresh=20)
df_bl_bcancer

,Id,Age,Bl_basophils number (1000 cells/ul),Bl_basophils percent (%),Bl_eosinophils number (1000 cells/ul),Bl_eosinophils percent (%),Bl_hematocrit (%),Bl_hemoglobin (g/dl),Bl_lymphocyte number (1000 cells/ul),Bl_lymphocyte percent (%),...,Bl_mean platelet volume (fl),Bl_monocyte number (1000 cells/ul),Bl_monocyte percent (%),Bl_platelet count (1000 cells/ul),Bl_red blood cell count (million cells/ul),Bl_red cell distribution width (%),Bl_segmented neutrophils num (1000 cell/ul),Bl_segmented neutrophils percent (%),Bl_white blood cell count (1000 cells/ul),Breast cancer
1,94.0,61.0,5.397605e-79,0.6,0.1,0.9,35.6,11.6,2.7,43.9,...,8.6,0.6,9.3,322.0,3.84,12.9,2.8,45.3,6.1,1
2,206.0,50.0,5.397605e-79,0.7,0.1,1.6,36.7,12.0,1.9,33.8,...,7.3,0.3,5.7,301.0,4.27,13.4,3.3,58.3,5.6,1
3,237.0,81.0,5.397605e-79,0.6,0.1,1.9,41.8,14.3,2.2,34.3,...,8.2,0.6,9.8,287.0,4.53,12.3,3.4,53.3,6.4,1
4,389.0,81.0,1.000000e-01,1.1,0.2,3.7,38.7,13.3,1.1,21.7,...,8.7,0.5,10.9,191.0,4.16,13.3,3.1,62.7,4.9,1
5,481.0,63.0,1.000000e-01,0.8,0.3,2.3,38.1,12.9,4.3,31.9,...,9.5,0.8,5.6,210.0,4.11,11.5,8.0,59.4,13.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,93308.0,80.0,1.000000e-01,0.9,0.5,4.8,41.4,13.8,2.6,23.3,...,10.1,0.7,6.4,215.0,4.62,15.1,7.1,64.7,11.0,1
78,93374.0,50.0,1.000000e-01,1.9,0.4,6.8,34.2,11.0,1.9,34.2,...,8.3,0.3,4.7,239.0,4.33,15.4,2.9,52.5,5.6,1
79,93413.0,57.0,1.000000e-01,1.0,0.1,0.9,42.5,13.8,1.8,23.8,...,8.0,0.6,8.0,238.0,4.39,12.8,5.0,66.4,7.5,1
80,93568.0,46.0,1.000000e-01,0.8,0.1,0.9,44.6,15.0,2.8,33.6,...,7.7,0.5,6.3,195.0,4.98,12.4,4.8,58.5,8.2,1


In [27]:
df_bl_bcancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 81
Data columns (total 21 columns):
Id                                             583 non-null float64
Age                                            583 non-null float64
Bl_basophils number (1000 cells/ul)            583 non-null float64
Bl_basophils percent (%)                       583 non-null float64
Bl_eosinophils number (1000 cells/ul)          583 non-null float64
Bl_eosinophils percent (%)                     583 non-null float64
Bl_hematocrit (%)                              583 non-null float64
Bl_hemoglobin (g/dl)                           583 non-null float64
Bl_lymphocyte number (1000 cells/ul)           583 non-null float64
Bl_lymphocyte percent (%)                      583 non-null float64
Bl_mean cell volume (fl)                       583 non-null float64
Bl_mean platelet volume (fl)                   583 non-null float64
Bl_monocyte number (1000 cells/ul)             583 non-null float64
Bl_monocyte 

In [28]:
df_bl_nbcancer

,Id,Age,Bl_basophils number (1000 cells/ul),Bl_basophils percent (%),Bl_eosinophils number (1000 cells/ul),Bl_eosinophils percent (%),Bl_hematocrit (%),Bl_hemoglobin (g/dl),Bl_lymphocyte number (1000 cells/ul),Bl_lymphocyte percent (%),...,Bl_mean platelet volume (fl),Bl_monocyte number (1000 cells/ul),Bl_monocyte percent (%),Bl_platelet count (1000 cells/ul),Bl_red blood cell count (million cells/ul),Bl_red cell distribution width (%),Bl_segmented neutrophils num (1000 cell/ul),Bl_segmented neutrophils percent (%),Bl_white blood cell count (1000 cells/ul),Breast cancer
0,6.0,19.0,5.397605e-79,5.397605e-79,2.000000e-01,2.0,38.7,13.1,1.9,19.4,...,7.7,0.3,3.4,453.0,4.22,12.8,7.2,75.3,9.6,0
1,7.0,59.0,5.397605e-79,7.000000e-01,5.397605e-79,0.8,40.2,13.4,2.3,46.8,...,8.2,0.4,8.5,244.0,4.60,14.3,2.1,43.3,4.9,0
2,15.0,38.0,5.397605e-79,6.000000e-01,1.000000e-01,1.0,40.2,13.3,1.6,20.8,...,7.7,0.4,5.6,255.0,4.14,11.9,5.5,72.0,7.6,0
3,16.0,85.0,5.397605e-79,5.000000e-01,5.397605e-79,0.6,33.3,10.9,2.1,28.0,...,8.0,0.5,6.3,219.0,3.57,14.1,4.8,64.6,7.4,0
4,20.0,23.0,5.397605e-79,2.000000e-01,1.000000e-01,1.0,41.2,14.5,1.6,28.1,...,8.0,0.5,9.1,220.0,4.53,11.4,3.4,61.6,5.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,93686.0,13.0,1.000000e-01,1.300000e+00,3.000000e-01,3.7,38.1,12.7,2.6,29.4,...,8.8,0.6,6.8,312.0,4.44,12.9,5.3,59.0,9.0,0
3385,93687.0,16.0,5.397605e-79,7.000000e-01,2.000000e-01,3.7,40.2,13.2,3.0,49.9,...,8.3,0.5,8.3,288.0,4.50,12.5,2.3,37.5,6.0,0
3386,93689.0,69.0,1.000000e-01,9.000000e-01,3.000000e-01,4.3,39.3,13.1,2.0,29.4,...,9.0,0.4,5.3,211.0,4.44,13.5,4.1,60.2,6.8,0
3387,93697.0,80.0,5.397605e-79,8.000000e-01,2.000000e-01,3.8,36.2,11.6,2.2,45.9,...,8.1,0.6,12.5,172.0,4.44,16.9,1.7,37.1,4.7,0


In [29]:
# remove the NaNs from the set without breast cancer
df_bl_nbcancer = df_bl_nbcancer.dropna(thresh=20)

# there are 583 entries, subset random 583 from the women that were not diagnosed
df_bl_nbcancer = df_bl_nbcancer.sample(n=583)

In [30]:
df_bl_nbcancer.isna().sum()

Id                                             0
Age                                            0
Bl_basophils number (1000 cells/ul)            0
Bl_basophils percent (%)                       0
Bl_eosinophils number (1000 cells/ul)          0
Bl_eosinophils percent (%)                     0
Bl_hematocrit (%)                              0
Bl_hemoglobin (g/dl)                           0
Bl_lymphocyte number (1000 cells/ul)           0
Bl_lymphocyte percent (%)                      0
Bl_mean cell volume (fl)                       0
Bl_mean platelet volume (fl)                   0
Bl_monocyte number (1000 cells/ul)             0
Bl_monocyte percent (%)                        0
Bl_platelet count (1000 cells/ul)              0
Bl_red blood cell count (million cells/ul)     0
Bl_red cell distribution width (%)             0
Bl_segmented neutrophils num (1000 cell/ul)    0
Bl_segmented neutrophils percent (%)           0
Bl_white blood cell count (1000 cells/ul)      0
Breast cancer       

In [31]:
# check for duplicates
df_bl.duplicated().sum() , df_bio.duplicated().sum()

(0, 0)

In [32]:
# create final datasets
df_bl = df_bl_nbcancer.append(df_bl_bcancer).sort_values(by=['Id'])
df_bl.to_csv('nhanes_breast_cancer_all_nhanes_blood.csv', index=False)
df_bio = df_bio_nbcancer.append(df_bio_bcancer).sort_values(by=['Id'])
df_bio.to_csv('nhanes_breast_cancer_all_nhanes_biochemistry.csv', index=False)